In [3]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pdb
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.feature_selection import RFECV

scipy: 0.19.1
numpy: 1.12.1
matplotlib: 2.0.2
pandas: 0.20.3
sklearn: 0.19.0


In [92]:
pd.set_option('display.max_columns', 300)

In [4]:
with open('../cspython/esl_teams.pkl', 'rb') as f: 
     d = pkl.load(f)

In [5]:
#with open('../cspython/test.pkl', 'rb') as f: 
#     d = pkl.load(f)

In [221]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [7]:
def combine_dfs(overview, big_data):
    first=True
    cols = ['map','round_num','half','match_id','series_id','ending','CT','T','side_winner','winner','team_A','team_B','team_A_score','team_B_score','match_num','team_players','K-D','+/-','ADR','KAST','Rating2.0','nicknames']
    dfs = []
    for idx, series_data in big_data.iteritems():
        series_data_m = merge_matches(series_data)
        series_data_m.loc[:,"date"] = overview.loc[overview.id == idx, 'date'].values[0]
        series_data_mo = merge_overview(series_data_m, series_data)
        series_data_mos = merge_scoreboards(series_data_mo, series_data)
        series_data_mosm = match_data_board_changer(series_data_mos, series_data)
        dfs.append(series_data_mosm)    
        new_cols = list(set(series_data_mosm.columns) -set(cols))
        cols += new_cols
    data = pd.concat(dfs)
    data = data.loc[:,cols]
    data = data.reset_index()
    return data

In [8]:
def merge_matches(series_data):
    for d in range(0,len(series_data['matches'])):
        if d == 0:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = series_data['matches'][d]
        else:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = pd.concat([series_data_m, series_data['matches'][d]])
    
    return series_data_m
    #should work for concact the matches together


In [182]:
def merge_overview(series_data_m, series_data):
    series_data['match_overview'].loc[:,'team_A_name'] = series_data['match_overview'].columns[4]
    series_data['match_overview'].loc[:,'team_B_name'] = series_data['match_overview'].columns[5]
    
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] == series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_B_name
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] != series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_A_name
    series_data['match_overview'] = series_data['match_overview'].rename(index = str, columns ={series_data['match_overview'].columns[3]: "winner_of_match",series_data['match_overview'].columns[4]: "team_A_score",series_data['match_overview'].columns[5]: "team_B_score"})
    
    
    series_data_mo = pd.merge(series_data_m, series_data['match_overview'], on=['match_id', 'map', 'series_id'])
    
    return series_data_mo
#works at merging matches with match_overviewb

In [220]:
def merge_scoreboards(series_data_mo, series_data):    
   
    for i in range(len(series_data['scoreboards'][0])):
        series_data['scoreboards'][0][i]['match_num'] = i+1
        #pdb.set_trace()
        series_data['scoreboards'][0][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_A_name'].unique()[0]
        series_data['scoreboards'][0][i] = series_data['scoreboards'][0][i].rename(index = str, columns={ series_data['scoreboards'][0][i].columns[0] : "team_players" })
        
        series_data['scoreboards'][1][i]['match_num'] = i+1
        series_data['scoreboards'][1][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_B_name'].unique()[0]
        series_data['scoreboards'][1][i] = series_data['scoreboards'][1][i].rename(index = str, columns={ series_data['scoreboards'][1][i].columns[0] : "team_players"})
        
        new_df = pd.concat([series_data['scoreboards'][0][i], series_data['scoreboards'][1][i]])
        
        if i == 0:
            con_df = new_df
        else:
            con_df = pd.concat([con_df, new_df])
   
    series_data_mos = pd.merge(series_data_mo, con_df, how='outer', on='match_num')
    series_data_mos['nicknames'] = series_data_mos['team_players'].str.split(pat = "'",expand = True)[1]
    return series_data_mos

       
        
#works at adding match_num to scoreboards
        
                         

In [11]:
def match_data_board_changer(series_data_mos,series_data):
    
    board_name = ['first_kills','who_kill_who', 'awp_kills']

    for idx, a in enumerate(series_data['match_data']):
        new_df = pd.DataFrame()
        for idx1, c in enumerate(board_name):    
            new_board_c = pd.DataFrame()
            new_board_r= pd.DataFrame()
            names_c = a[c].set_index('Unnamed: 0').columns
            for b in names_c:
                new_board_c[b+'_'+c] = a[c].set_index('Unnamed: 0')[b].str.split(pat = ':', expand = True)[0]
               
            names_r = a[c].set_index('Unnamed: 0').T.columns
            for b in names_r:
                new_board_r[b+'_'+c] = a[c].set_index('Unnamed: 0').T[b].str.split(pat = ':', expand = True)[1]
            new_board_c['nicknames'] = new_board_c.index
            new_board_r['nicknames'] = new_board_r.index
            
            board_df = new_board_c.append(new_board_r)
            
            if idx1 == 0:
                new_df = board_df
            else:
                new_df = pd.merge(new_df, board_df, on = 'nicknames')
        if idx == 0:
            new_df['match_num'] = 1+idx
            con_df = new_df
        else:
            new_df['match_num'] = 1+idx
            try:
                con_df = con_df.append(new_df, ignore_index=True)
            except:
                print con_df.columns
                print new_df.columns
    con_df = con_df.loc[:, ~con_df.columns.duplicated()]
    #pdb.set_trace()
  
    series_data_mosm = pd.merge(series_data_mos, con_df, on=['nicknames','match_num'])
    return series_data_mosm


In [222]:
#data = combine_dfs(overview, big_data)

In [226]:
#data.to_pickle("data_converged.pkl")


In [873]:
data = pd.read_pickle("data_converged.pkl")

In [705]:
def create_opponent_team_col(data):
    data.loc[:,'player_team_opponent'] = np.nan
    data.loc[(data['team_A_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_A_name']
    data.loc[(data['team_B_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_B_name']
    return data
    

In [706]:
def create_fwa_dr_columns(data, col_list):  # first k , awp, who, divided by rounds 
    columns = pd.Series(data.columns)
    for a in col_list:
        col = columns[columns.str.contains(a)]
        data[a+'_sum_dr'] = data[col].convert_objects(convert_numeric = True).sum(axis = 1) / (data['team_A_score'] + data['team_B_score'])*100
    data.loc[:, data.columns != 'date'] = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    return data
        
    

In [707]:
def create_player_columns(data):
    names = data.nicknames.unique()
    for a in names:
        data.loc[:,a] = 0
        data.loc[data.loc[:,'nicknames'] == a, a] = 5   # its 5 so that when you group by team it becomes 1 
    return data
    

In [708]:
def apply_nummeric_and_group_as_match(data):
    r = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    r['date'] = data.date
    data = r
    data = data.fillna(0)
    data_match = data.groupby(['match_id', 'player_team_name', 'date', 'team_A_name', 'team_B_name', 'series_id', 'map', 'winner_of_match', 'loser_of_match','player_team_opponent']).mean()
    data_match = pd.DataFrame(data_match)
    data_match = data_match.reset_index()
    return data_match

In [709]:
col_list = ['first_kills', 'who_kill_who','awp_kills']
data = create_opponent_team_col(data)
data = create_fwa_dr_columns(data, col_list)
data = create_player_columns(data)
data = apply_nummeric_and_group_as_match(data)

In [710]:
#data.to_pickle("data_converged_historic.pkl")


In [903]:
data = pd.read_pickle("data_converged_historic.pkl")

In [904]:
def create_historic_data(data):
    col_list = ['first_kills', 'who_kill_who','awp_kills']
    data = create_fwadr_his(data, col_list)
    data = create_matches_count(data)
    data = create_avdamage_his(data)
    data = create_avdamage_map_his(data)
    data = create_faw_map_his(data, col_list)
    return data
        
        

In [905]:
def create_matches_count(data): # how many matches a team has played
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].count()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).sum()
        grouping = grouping.rename(index=str, columns={'ADR': 'matches_played_team'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data
    
    

In [906]:
def create_avdamage_his(data):  # column with historic average damage of individual !!!
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
        grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data
   
  

In [907]:
def create_avdamage_map_his(data):# historic average damage of individual for each map !!!
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    new_group = pd.DataFrame()
    for a in maps:
        for b in teams:   
            data_team = data.loc[(data.loc[:,'player_team_name'] == b) & (data.loc[:,'map'] == a), ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','match_id'])['ADR'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist_on_map'})
            new_group = pd.concat([new_group, grouping])
    
    data = pd.merge(data, new_group, on = ['match_id','player_team_name'])

    return data



In [908]:
def create_fwadr_his(data, col_list):
    teams = list(data.player_team_name.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for a in teams:   
            data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','date','match_id'])[b+'_sum_dr'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist'})
            new_group = pd.concat([new_group, grouping])
        data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])
    return data

In [909]:
def create_faw_map_his(data, col_list):
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for i in maps:
            for a in teams:   
                data_team = data.loc[(data.loc[:,'player_team_name'] == a) & (data.loc[:,'map'] == i), ].sort_values(by='date',ascending=True)
                grouping = data_team.groupby(['player_team_name','match_id'])[b+'_sum_dr'].max()
                grouping = pd.DataFrame(grouping)
                grouping = grouping.reset_index()
                grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
                grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist_on_map'})
                new_group = pd.concat([new_group, grouping])
                
        data = pd.merge(data, new_group, on = ['match_id','player_team_name'])
    return data   


In [910]:
data = create_historic_data(data)

In [ ]:
data.loc[(data.loc[:,'player_team_name'] == 'Cloud9') & (data.loc[:,'map'] == 'Inferno'),].sort_values(by='date',ascending=True)

In [ ]:
#  FUNCTIONS BELOW HERE NEED TO BE WORKED ON

In [50]:
def create_map_win_loss_his(data):  # team total win and loses on map with total times played on map !4!
    maps = data.map.unique()
    match_id = data.match_id.unique()
    for a in maps:
        data[a + "_win_his"] = 0
        data[a + "_loss_his"] = 0
        data[a + "_total_played"] = 0
    for a in match_id:
        map_for_match = data.loc[(data['match_id'] == a) ,'map'].unique()
        winner_of_map = data.loc[(data['match_id'] == a), 'winner_of_match'].unique()
        loser_of_map = data.loc[(data['match_id'] == a), 'loser_of_match'].unique()
        data.loc[(data['player_team_name'] == winner_of_map[0]), [map_for_match[0] + "_win_his", map_for_match[0] +'_total_played']] += 1
        data.loc[(data['player_team_name'] == loser_of_map[0]), [map_for_match[0] + "_loss_his", map_for_match[0] +'_total_played']] += 1 
       
    return data

In [223]:
def create_map_win_his_per(data): #percentage team total win and loses on map !5!
    teams = data.player_team_name.unique()
    maps = data.map.unique()
    for a in maps:
        data[a + '_win_perc_map'] = 0
        for b in teams:
            pg = (data.player_team_name == b) 
            data.loc[pg,a + '_win_perc_map'] = data.loc[pg, a + "_win_his"].unique()[0] / float((data.loc[pg, a + "_win_his"].unique()[0] + data.loc[pg, a + "_loss_his"].unique()[0])) * 100 
    data = data.fillna(0)        
    return data    
    
    

In [224]:
def create_rounds_won_vs_team_his(data): # team rounds won vs another team    !6!
    grouping = data.groupby(['match_id','map','team_A_name','team_B_name', 'team_A_score', 'team_B_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['team_A_name', 'team_B_name']).sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    forward = grouping.team_A_name+grouping.team_B_name
    reverse = grouping.team_B_name+grouping.team_A_name
    for idx, val in enumerate(forward):
        for idx2, val2 in enumerate(reverse):
            if val == val2 and idx < idx2:
                grouping.loc[idx,'team_A_score_Count'] += grouping.loc[idx2,'team_B_score_Count']
                grouping.loc[idx,'team_B_score_Count'] += grouping.loc[idx2,'team_A_score_Count']
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
            elif val == val2 and idx > idx2:
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
    
    grouping = grouping.drop('round_num_Count_Count', axis = 1)
    col1 = list(grouping.team_A_name.unique())
    col2  = list(grouping.team_B_name.unique())
    col = col1 + col2
    col = list(set(col))
    data = pd.merge(data,grouping, on=['team_A_name', 'team_B_name']) 
    for a in col:
        data['rd_total_his_'+ a] = 0
        data.loc[(data.player_team_name != a) & (data.team_A_name == a) , 'rd_total_his_'+ a]=data.team_B_score_Count
        data.loc[(data.player_team_name != a) & (data.team_B_name == a) , 'rd_total_his_'+ a]=data.team_A_score_Count
        bgrouping = data.groupby(['player_team_name'])['rd_total_his_'+ a].max()
        bgrouping = pd.DataFrame(bgrouping)
        bgrouping = bgrouping.reset_index()
        data = data.drop('rd_total_his_'+ a, axis = 1)
        data = pd.merge(data, bgrouping, on = 'player_team_name')
       
    return data    

In [225]:
def create_total_team_rd_map_his(data):
    grouping = data.groupby(['map','team_A_name', 'team_A_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['map','team_A_name'])[ 'team_A_score'].sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    fgrouping = data.groupby(['player_team_name','map','team_B_name', 'team_B_score'])['round_num'].count()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.groupby(['map','team_B_name'])[ 'team_B_score'].sum()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.rename(index=str, columns={"team_B_name": "team_A_name", 'team_B_score_Count': 'team_A_score_Count'})
    merged = pd.concat([grouping, fgrouping], axis = 0)
    merged.groupby(['map', 'team_A_name'])['team_A_score_Count'].sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index(drop = True)
    merged = merged.rename(index=str, columns={"team_A_name": "player_team_name", "team_A_score_Count": 'total_team_rd_map'})
    merged = merged.groupby(['player_team_name', 'map']).sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index()
    for a in list(merged.map.unique()):
        merged.loc[:,'total_team_rd_'+ a] = 0
        merged.loc[(merged.loc[:, 'map'] == a), 'total_team_rd_'+ a] = merged.loc[:,'total_team_rd_map']
        ok_map = merged.groupby(['player_team_name'])['total_team_rd_'+ a].max()
        ok_map = pd.DataFrame(ok_map)
        ok_map = ok_map.reset_index()
        data = pd.merge(data, ok_map, on = 'player_team_name')
    return data
    

In [ ]:
#FUNCTIONS ABOVE HERE NEED TO BE WORKED ON

In [230]:
#data.to_pickle("data_converged_changed.pkl")


In [229]:
col_list = ['first_kills', 'who_kill_who','awp_kills']
data = create_fwa_dr_columns(data, col_list)
data = create_fwadr_his(data,col_list)
data = create_avdamage_his(data)
data = create_map_win_loss_his(data)
data = create_map_win_his_per(data)
data = create_rounds_won_vs_team_his(data)
data = create_total_team_rd_map_his(data)
data = create_avdamage_map_his(data)
data = create_faw_map_his(data, col_list)
data = create_matches_count(data)
data = create_opponent_team_col(data)

In [99]:
data = pd.read_pickle("data_converged_changed.pkl")

In [100]:
data_adv = data.loc[:, 'first_kills_sum_dr_hist':'player_team_opponent']

In [101]:
data_adv['date'] = data['date']

In [102]:
data_adv['match_id'] = data['match_id']

In [103]:
data_adv['player_team_name'] = data['player_team_name']

In [104]:
data_adv['map'] = data['map']

In [105]:
data_adv['winner_of_match']= data['winner_of_match']

In [106]:
data_adv = data_adv.drop(['team_A_score_Count','team_B_score_Count'], axis = 1)

In [107]:
data_adv = data_adv.groupby(['match_id', 'player_team_name', 'player_team_opponent','map', 'winner_of_match', 'date']).mean()

In [108]:
data_adv = pd.DataFrame(data_adv)
data_adv = data_adv.reset_index()

In [109]:
data_adv = data_adv.sort_values('date')

In [110]:
data_adv = data_adv.drop(['Mirage_win_his','Mirage_loss_his','Train_win_his','Train_loss_his','Cobblestone_win_his'
                      ,'Cobblestone_loss_his','Cache_win_his','Cache_loss_his','Inferno_win_his','Inferno_loss_his'
                       ,'Overpass_win_his','Overpass_loss_his','Nuke_win_his','Nuke_loss_his','who_kill_who_Cache_dr_hist'
                       ,'who_kill_who_Cobblestone_dr_hist','who_kill_who_Inferno_dr_hist','who_kill_who_Mirage_dr_hist'
                       ,'who_kill_who_Nuke_dr_hist','who_kill_who_Overpass_dr_hist','who_kill_who_Train_dr_hist'
                       , 'who_kill_who_sum_dr_hist'], axis = 1)#'player_team_name_CLG','player_team_name_Cloud9', 'player_team_name_Ghost'
#                       ,'player_team_name_Immortals', 'player_team_name_Liquid','player_team_name_Luminosity'
#                       , 'player_team_name_Misfits','player_team_name_NRG', 'player_team_name_OpTic','player_team_name_Renegades'
#                       , 'player_team_name_Rogue','player_team_name_SK', 'player_team_name_Splyce','player_team_name_compLexity','map_Cache'
#                       ,'map_Cobblestone','map_Inferno','map_Mirage','map_Nuke','map_Overpass','map_Train'
#                       ,'player_team_opponent_CLG','player_team_opponent_Cloud9','player_team_opponent_Ghost'
#                       ,'player_team_opponent_Immortals','player_team_opponent_Liquid','player_team_opponent_Luminosity'
#                       ,'player_team_opponent_Misfits','player_team_opponent_NRG','player_team_opponent_OpTic'
#                       ,'player_team_opponent_Renegades','player_team_opponent_Rogue','player_team_opponent_SK'
#                       ,'player_team_opponent_Splyce','player_team_opponent_compLexity'], axis = 1)

In [111]:
data_adv_even = data_adv.iloc[::2]  # even
data_adv_odd = data_adv.iloc[1::2]


In [112]:
data_adv_merged = pd.merge(data_adv_even, data_adv_odd, on =['match_id', 'date'])

In [113]:
pd.set_option('display.max_columns', 300)

In [114]:
data_adv.shape

(1208, 66)

In [115]:
data_adv_merged.loc[:,data_adv_merged.columns != 'date'] = data_adv_merged.loc[:,data_adv_merged.columns != 'date'].apply(pd.to_numeric, errors='ignore')

In [116]:
datay = data_adv_merged.loc[:,['winner_of_match_x', 'player_team_name_x', 'match_id']]

In [117]:
data_adv_merged = pd.get_dummies(data_adv_merged,columns = ['map_x'])

In [118]:
data_adv_merged.shape

(544, 136)

In [119]:
datay.loc[datay.winner_of_match_x != datay.player_team_name_x, 'winner_of_match_x'] = 0
datay.loc[datay.winner_of_match_x == datay.player_team_name_x, 'winner_of_match_x'] = 1


In [120]:
datay.winner_of_match_x = datay.winner_of_match_x.apply(pd.to_numeric, errors='ignore')

In [121]:
datay = datay.drop(['player_team_name_x','match_id'], axis = 1)

In [122]:
data_adv_merged = data_adv_merged.drop(['match_id'], axis = 1)

In [123]:
for a in data_adv_merged:
    print a

player_team_name_x
player_team_opponent_x
winner_of_match_x
date
first_kills_sum_dr_hist_x
awp_kills_sum_dr_hist_x
ADR_hist_x
Train_total_played_x
Cobblestone_total_played_x
Mirage_total_played_x
Cache_total_played_x
Inferno_total_played_x
Overpass_total_played_x
Nuke_total_played_x
Train_win_perc_map_x
Cobblestone_win_perc_map_x
Mirage_win_perc_map_x
Cache_win_perc_map_x
Inferno_win_perc_map_x
Overpass_win_perc_map_x
Nuke_win_perc_map_x
rd_total_his_Ghost_x
rd_total_his_OpTic_x
rd_total_his_Liquid_x
rd_total_his_Luminosity_x
rd_total_his_CLG_x
rd_total_his_NRG_x
rd_total_his_Cloud9_x
rd_total_his_Immortals_x
rd_total_his_SK_x
rd_total_his_compLexity_x
rd_total_his_Rogue_x
rd_total_his_Misfits_x
rd_total_his_Splyce_x
rd_total_his_Renegades_x
total_team_rd_Cache_x
total_team_rd_Cobblestone_x
total_team_rd_Inferno_x
total_team_rd_Mirage_x
total_team_rd_Overpass_x
total_team_rd_Train_x
total_team_rd_Nuke_x
ADR_his_Cache_x
ADR_his_Cobblestone_x
ADR_his_Inferno_x
ADR_his_Mirage_x
ADR_his_Nu

In [134]:
data_adv_merged = data_adv_merged.round(4)

In [135]:
#data_adv_temp = data_adv.loc[:,[,'Train_win_perc_map','Cobblestone_win_perc_map','Mirage_win_perc_map'
#                                ,'Cache_win_perc_map','Inferno_win_perc_map','Overpass_win_perc_map','Nuke_win_perc_map']]

In [144]:
X = data_adv_merged

In [145]:
X = X.select_dtypes(include = ['int64', 'float64', 'uint8', 'bool','datetime64[ns]']).iloc[:, 1:]

In [128]:
y = datay.values.astype(int)
X = X.values

In [129]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [130]:
num_folds = 3
num_instances = len(X) 
seed = 7
scoring = 'accuracy'


In [131]:
#X = data_adv.iloc[:,top_33_important_features]

In [132]:
models = []
models.append(('LR', LogisticRegression(random_state = seed)))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('NB', GaussianNB()))
models.append(('KNeighborsClassifier', KNeighborsClassifier()))#ewights = 'distance' 
models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = seed)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = seed)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state = seed)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state = seed)))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.TimeSeriesSplit()
    cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.598039 (0.019299)
LDA: 0.588235 (0.021646)
NB: 0.666667 (0.029615)
KNeighborsClassifier: 0.578431 (0.035178)
XGBClassifier: 0.629902 (0.039974)
GradientBoostingClassifier: 0.620098 (0.009171)
AdaBoostClassifier: 0.598039 (0.012498)
RandomForestClassifier: 0.575980 (0.024263)
ExtraTreesClassifier: 0.605392 (0.063062)
DecisionTreeClassifier: 0.580882 (0.036519)


In [35]:
estimator = xgb.XGBClassifier()
rfe = RFECV(estimator,cv = kfold)
fit = rfe.fit(X,y)
print("Num of feature: %d") % fit.n_features_
#print("Selected features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

Num of feature: 33
Feature Ranking: [ 1  1  1  1  1  1 14  1  7  8 11 18  4  1  2  1  1 12  1  1  1 17  1 10  5
  6  1  1  1 15  1 13  3 19  1  1  1  1 20  1  1  1  1  1  1  1  1  1  9  1
  1 16]


In [36]:
top_33_important_features = [] 
for b in range(0,len(fit.ranking_)):
    if fit.ranking_[b] == 1:
        top_33_important_features.append(b)
        print b,data_adv.columns[b]

0 first_kills_sum_dr_hist
1 who_kill_who_sum_dr_hist
2 awp_kills_sum_dr_hist
3 ADR_hist
4 Train_win_his
5 Train_loss_his
7 Cobblestone_win_his
13 Cache_win_his
15 Cache_total_played
16 Inferno_win_his
18 Inferno_total_played
19 Overpass_win_his
20 Overpass_loss_his
22 Nuke_win_his
26 Cobblestone_win_perc_map
27 Mirage_win_perc_map
28 Cache_win_perc_map
30 Overpass_win_perc_map
34 rd_total_his_Liquid
35 rd_total_his_Luminosity
36 rd_total_his_CLG
37 rd_total_his_NRG
39 rd_total_his_Immortals
40 rd_total_his_SK
41 rd_total_his_compLexity
42 rd_total_his_Rogue
43 rd_total_his_Misfits
44 rd_total_his_Splyce
45 rd_total_his_Renegades
46 total_team_rd_Cache
47 total_team_rd_Cobblestone
49 total_team_rd_Mirage
50 total_team_rd_Overpass


In [1054]:
for a in range(0, len(models)):
    model1 = models[a]
    for b in range(a+1, len(models)):
        model2 = models[b]
        for c in range(b+1, len(models)):
            model3 = models[c]
            estimators = []
            estimators.append(model1)
            estimators.append(model2)
            estimators.append(model3)
            ensemble = VotingClassifier(estimators, voting='soft')
            results = model_selection.cross_val_score(ensemble, X, y, cv=kfold, scoring= scoring)
            if results.mean() > .69:
                print(model1[0], model2[0], model3[0],results.mean(), results.std())

KeyboardInterrupt: 